In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import sys 
sys.path.append("..")

In [ ]:
import numpy as np
import pandas as pd
from trackreid.reid_processor import ReidProcessor
import cv2
from tqdm import tqdm 


# Real life data

In [ ]:
import os
from lib.sequence import Sequence
from bytetracker import BYTETracker
from lib.bbox.utils import xy_center_to_xyxy, rescale_bbox

In [ ]:
DATA_PATH = "../data"
DETECTION_PATH = f"{DATA_PATH}/detections"
FRAME_PATH = f"{DATA_PATH}/frames"
VIDEO_OUTPUT_PATH = "private"

SEQUENCES = os.listdir(FRAME_PATH)


In [ ]:
def get_sequence_frames(sequence):
    frames = os.listdir(f"{FRAME_PATH}/{sequence}")
    frames = [os.path.join(f"{FRAME_PATH}/{sequence}", frame) for frame in frames]
    frames.sort()
    return frames

def get_sequence_detections(sequence):
    detections = os.listdir(f"{DETECTION_PATH}/{sequence}")
    detections = [os.path.join(f"{DETECTION_PATH}/{sequence}", detection) for detection in detections]
    detections.sort()
    return detections

frame_path = get_sequence_frames(SEQUENCES[2])
test_sequence = Sequence(frame_path)
test_sequence

In [ ]:
class DetectionHandler():
    def __init__(self, image_shape) -> None:
        self.image_shape = image_shape

    def process(self, detection_output):
        if detection_output.size:
            if detection_output.ndim == 1:
                detection_output = np.expand_dims(detection_output, 0)

            processed_detection = np.zeros(detection_output.shape)

            for idx, detection in enumerate(detection_output):
                clss = detection[0]
                conf = detection[5]
                bbox = detection[1:5]
                xyxy_bbox = xy_center_to_xyxy(bbox)
                rescaled_bbox = rescale_bbox(xyxy_bbox,self.image_shape)
                processed_detection[idx,:4] = rescaled_bbox
                processed_detection[idx,4] = conf
                processed_detection[idx,5] = clss
                

            return processed_detection
        else:
            return detection_output
            
detection_handler = DetectionHandler(image_shape=[2560, 1440])

In [ ]:
class TrackingHandler():
    def __init__(self) -> None:
        pass
    def process(self, tracked_objects : np.ndarray, frame_id):

        if tracked_objects.size :
            if tracked_objects.ndim == 1:
                tracked_objects = np.expand_dims(tracked_objects, 0)

            processed_tracked = np.zeros((len(tracked_objects), 8))
            processed_tracked[:,0] = frame_id
            processed_tracked[:,1] = tracked_objects[:,4]
            processed_tracked[:,2] = tracked_objects[:,5]
            processed_tracked[:,3:7] = tracked_objects[:,:4]
            processed_tracked[:,7] = tracked_objects[:,6]
        
            return processed_tracked
        else:
            return tracked_objects

        
tracking_handler = TrackingHandler()

In [ ]:
def bounding_box_distance(obj1, obj2):
    # Get the bounding boxes from the Metadata of each TrackedObject
    bbox1 = obj1.metadata.bbox
    bbox2 = obj2.metadata.bbox

    # Calculate the Euclidean distance between the centers of the bounding boxes
    center1 = ((bbox1[0] + bbox1[2]) / 2, (bbox1[1] + bbox1[3]) / 2)
    center2 = ((bbox2[0] + bbox2[2]) / 2, (bbox2[1] + bbox2[3]) / 2)
    distance = np.sqrt((center1[0] - center2[0])**2 + (center1[1] - center2[1])**2)

    return distance

# TODO : discard by zone
def select_by_category(obj1, obj2):
    # Compare the categories of the two objects
    return 1 if obj1.category == obj2.category else 0

In [ ]:
frame_id = 0
from bytetracker.basetrack import BaseTrack
BaseTrack._count = 0

# Define the codec using VideoWriter_fourcc() and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'avc1')  # or use 'x264'
out = cv2.VideoWriter('output_corrected.mp4', fourcc, 20.0, (2560, 1440))  # adjust the frame size (640, 480) as per your needs


tracker = BYTETracker(track_thresh= 0.3, track_buffer = 5, match_thresh = 0.85, frame_rate= 30)
reid_processor = ReidProcessor(filter_confidence_threshold=0.1, 
                          filter_time_threshold=0,
                          cost_function=bounding_box_distance,
                          selection_function=select_by_category,
                          max_attempt_to_rematch=1,
                          max_frames_to_rematch=100)



for frame, detection in tqdm(test_sequence):
    frame = np.array(frame)

    processed_detections = detection_handler.process(detection)
    if processed_detections.size:
        frame_id += 1
        tracked_objects = tracker.update(dets=processed_detections,_=None)
        processed_tracked = tracking_handler.process(tracked_objects, frame_id)
        if processed_tracked.size:
            result = reid_processor.update(processed_tracked, frame_id)
        else:
            result = []
    else:
        result = []

    stop = False

    if result:
        for res in result:
            frame_id, object_id, class_id, x1, y1, x2, y2, confidence_score = res
            frame_id, object_id, class_id, x1, y1, x2, y2 = int(frame_id), int(object_id), int(class_id), int(x1), int(y1), int(x2), int(y2)
            color = (0, 255, 0) if class_id == 0.0 else (0, 0, 255)  # green for class 0, red for class 1
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, str(object_id), (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
            if object_id == 34 :
                break
                stop = True
    
    if stop :
        break


    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Write the frame to the video file
    out.write(frame)
out.release()

In [ ]:
reid_processor.all_tracked_objects